# Everytime you restart container, you do better close the browser and restart JupyterLab UI again to avoid the issues caused by caching!!!

In [1]:
import os
import yaml
import nbimporter
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp
from lab_table_manager import TableManager
from lab_schema_manager import SchemaManager
import yfinance as yf
from lab_pg_database_manager import PGDatabaseManager
from lab_spark import create_spark_session
from lab_raw_yfinance import RawYFinance

In [2]:
def fetch_yfinance_record(symbol_date_pairs):
    try:
        symbol, start_date = symbol_date_pairs
        # Fetch stock data using yfinance
        quote = yf.Ticker(symbol)
        current_date = date.today()
        hist = quote.history(start=start_date, end=current_date)

        # Reset index to include Date as a column and format it
        hist.reset_index(inplace=True)
        hist["Date"] = hist["Date"].dt.date
        
        # limit and stablize the fields of hist
        hist = hist[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]
        
        # Add symbol and import_time to each row
        record_list = [
            tuple(row) + (symbol, import_time) for row in hist.itertuples(index=False)
        ]

        # print(record_list)
        return record_list

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return []  # Return an empty list on error

In [3]:
def parallel_fetch_yfinance_record(symbol_date_pairs, record_schema):
    try:
        # Distribute (symbol, start_date) pairs across Spark workers
        record_rdd = spark.sparkContext.parallelize(symbol_date_pairs)
        
        # Fetch data in parallel
        mapped_record_rdd = record_rdd.flatMap(fetch_yfinance_record)

        # Convert RDD to DataFrame
        result_df = spark.createDataFrame(mapped_record_rdd, schema=record_schema)
        
        return result_df
    except Exception as e:
        print(f"Error paralleling fetch: {e}")
        return spark.createDataFrame([])

In [4]:
def load_iceberg_raw_eod_yfinance(symbol_date_pairs, iceberg_sink_table, schema_config_file):
    try: 
        schema_manager=SchemaManager(schema_config_file)
        schema_struct_type=schema_manager.get_struct_type("tables", iceberg_sink_table)  
        # print(schema_struct_type)
        
        create_table_script = schema_manager.get_create_table_query("tables", iceberg_sink_table)
        # print(create_table_script)
        spark.sql(create_table_script)

        df_source=parallel_fetch_yfinance_record(symbol_date_pairs, schema_struct_type)        
        # df_source.writeTo(iceberg_sink_table).append()
        df_source.write.mode("overwrite").saveAsTable(iceberg_sink_table)    

        print(f"{iceberg_sink_table} has been loaded")
        
    except Exception as e:
        print(f"Error loading lceberg raw table: {e}")
        

In [5]:
def load_pg_finalytics(iceberg_source_table, pg_url, pg_driver, pg_sink_table, pg_truncate_script, pg_merge_script):   
    try:     
        df_source=spark.read.table(iceberg_source_table)          
        finalytics.execute_sql_script(pg_truncate_script)
        
        # Write DataFrame to PostgreSQL
        df_source.write.jdbc(url=pg_url, table=pg_sink_table, mode="append", properties={"driver": pg_driver})        
        finalytics.execute_sql_script(pg_merge_script)
        
    except Exception as e:
        print(f"Error loading pg finalytics: {e}")        


## Load nessie.raw.stock_eod_yfinance

In [6]:
# symbol_start_date_pairs = [
#     ('TSLA', '2024-11-20'),
#     ('AAPL', '2024-11-20'),
#     ('C', '2024-11-20'),
# ]

# Get finalytics db connetion info
conn_config_file='cfg_connections.yaml'
finalytics=PGDatabaseManager(conn_config_file, 'finalytics')
pg_url=finalytics.jdbc_url
pg_driver=finalytics.driver

# Get symbol_start_date_pairs from finalytics db
query="select symbol, start_date from fin.vw_etl_stock_eod_start_date limit 20"
# query="select symbol, start_date from fin.vw_etl_stock_eod_start_date"
symbol_start_date_pairs=finalytics.get_sql_script_result_list(query)
# print(symbol_start_date_pairs)


# Get iceberg table config info
table_schema_config_file='cfg_table_schemas.yaml'
iceberg_table='nessie.raw.stock_eod_yfinance'

# Set global import_time
import_time = datetime.now().isoformat()


# Create Spark Session
connection_config_file="cfg_connections.yaml"
spark_app_name="raw_yfinance"
spark=create_spark_session(connection_config_file, spark_app_name)
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.raw;")

# Load nessie.raw.stock_eod_yfinance 
load_iceberg_raw_eod_yfinance(symbol_start_date_pairs, iceberg_table, table_schema_config_file)


:: loading settings :: url = jar:file:/opt/workspace/finalytics_spark/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f0dd957b-1aeb-4445-8ab9-13fe8012183e;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.0 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.77.1 in central
	found software.amazon.awssdk#bundle;2.29.42 in central
	found software.amazon.awssdk#url-connection-client;2.29.42 in central
	found software.amazon.awssdk#utils;2.

nessie.raw.stock_eod_yfinance has been loaded


## Load finalytics.stage.stock_eod_quote_yahoo and merge into fin.stock_eod_quote

In [7]:
pg_table = "stage.stock_eod_quote_yahoo"  # Replace with the PostgreSQL table name

pg_truncate_script = f"truncate table {pg_table};"

pg_merge_script = "call fin.usp_load_stock_eod();"

load_pg_finalytics(iceberg_table, pg_url, pg_driver, pg_table,  pg_truncate_script, pg_merge_script)

In [8]:
spark.sql(f'select count(*) from {iceberg_table}').show()

+--------+
|count(1)|
+--------+
|      37|
+--------+



In [9]:
spark.sql('select * from nessie.raw.stock_eod_yfinance').show(truncate=False)

+----------+-----+-----+------+-----+--------+---------+------------+------+--------------------------+
|date      |open |high |low   |close|volume  |dividends|stock_splits|symbol|import_time               |
+----------+-----+-----+------+-----+--------+---------+------------+------+--------------------------+
|2024-11-14|1.9  |1.725|1.725 |1.725|129682  |0.0      |0.0         |EIGR  |2025-01-03T03:01:30.433839|
|2024-12-31|23.54|24.25|23.48 |23.76|956700  |0.0      |0.0         |GDS   |2025-01-03T03:01:30.433839|
|2025-01-02|23.57|23.85|23.1  |23.18|723400  |0.0      |0.0         |GDS   |2025-01-03T03:01:30.433839|
|2024-12-31|1.9  |1.91 |1.89  |1.91 |10097100|0.0      |0.0         |BBD   |2025-01-03T03:01:30.433839|
|2025-01-02|1.87 |2.0  |1.86  |1.96 |63000700|0.0      |0.0         |BBD   |2025-01-03T03:01:30.433839|
|2024-12-31|3.5  |3.52 |3.26  |3.28 |1220800 |0.0      |0.0         |DDL   |2025-01-03T03:01:30.433839|
|2025-01-02|3.28 |3.289|3.116 |3.24 |1147600 |0.0      |0.0     

In [10]:
# print(symbol_start_date_pairs)

[('EIGR', '2024-11-15'), ('NYCB', '2024-12-27'), ('GDS', '2024-12-31'), ('BBD', '2024-12-31'), ('DDL', '2024-12-31'), ('WTMAU', '2024-12-31'), ('SNT', '2024-12-31'), ('ASTC', '2024-12-31'), ('TWIN', '2024-12-31'), ('SONX', '2024-12-31'), ('VITL', '2024-12-31'), ('INGN', '2024-12-31'), ('MAQCU', '2024-12-31'), ('GRMN', '2024-12-31'), ('RVLV', '2024-12-31'), ('FVCB', '2024-12-31'), ('ETJ', '2024-12-31'), ('CTSH', '2024-12-31'), ('MMC', '2024-12-31'), ('BBIG', '2024-12-31')]
